# Overview

Download population density geotiff from https://hub.worldpop.org/geodata/summary?id=43847


In [1]:
import atlite
import xarray as xr
import pandas as pd
import PySAM.Pvwattsv8 as pv
import numpy as np
import os
import matplotlib.pyplot as plt
import rasterio
from _helpers import (
    get_nsrdb_weather_file,
    aggregate_intra_region,
    generate_pv_timeseries,
    reshape_xarray,
)
from rasterio.plot import show
from rasterio.features import geometry_mask

from _helpers import load_gis_data, set_availability
import geopandas as gpd
from IPython.display import clear_output
from scipy.spatial.distance import cdist
from shapely import wkt
import concurrent.futures

import pyogrio
from shapely.geometry import Point

In [2]:
albers_crs = {
    "proj": "aea",  # Albers Equal Area
    "lat_1": 29.5,  # First standard parallel
    "lat_2": 45.5,  # Second standard parallel
    "lat_0": 23,    # Latitude of projection center
    "lon_0": -96,   # Longitude of projection center
    "x_0": 0,
    "y_0": 0,
    "datum": "NAD83",
    "units": "m",
    "no_defs": True
}

In [4]:
buildings = pyogrio.read_dataframe(
    "/media/pklein/1TB_SSD/Meridian Dropbox/ME Projects/03 R E S O U R C E S/Modelling data/GIS data/Buildings/hotosm_zaf_buildings_polygons_shp/hotosm_zaf_buildings_polygons.shp"
)
buildings = buildings.to_crs(albers_crs)
buildings.sindex
buildings["area"] = buildings.geometry.area

In [ ]:
bin = bin.groupby(["y","x"]).sum()["area"]

In [5]:
cutout = atlite.Cutout(
    path="../../cutouts/RSA-2010_22-era5.nc",
    chunks={'time': 100}
)
grid = cutout.grid
grid = grid.to_crs(albers_crs)

In [6]:
buildings_in_cells = gpd.sjoin(buildings, grid, op="within")

/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [7]:
data_bundle_path = "../../data/bundle"
gis_data = load_gis_data(data_bundle_path)

Loading Eskom Supply Regions from ../../data/bundle/rsa_supply_regions.gpkg
Loading EIA applications from ../../data/bundle/REEA_OR_2023_Q3.shp
Loading REDZs from ../../data/bundle/REDZs.shp
Loading Power Corridors from ../../data/bundle/Power_corridors.shp
Loading SACAD from ../../data/bundle/SACAD_OR_2023_Q3.shp
Loading SAPAD from ../../data/bundle/SAPAD_OR_2023_Q3.shp
Loading SKA exclusion from ../../data/bundle/SKA/SKA_exclusion.shp


In [20]:
building_area_in_cell = buildings_in_cells.groupby(["y","x"]).sum()["area"].to_xarray()

/tmp/ipykernel_30297/1947448932.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  building_area_in_cell = buildings_in_cells.groupby(["y","x"]).sum()["area"].to_xarray()


In [29]:
for region_num in gis_data["supply_regions"].keys():
    print(region_num)
    bus_list = gis_data["supply_regions"][region_num].name.to_list()
    map_cell_to_region = gpd.sjoin(grid.to_crs(albers_crs), gis_data["supply_regions"][region_num].to_crs(albers_crs), op="within")[["x", "y", "name"]]

    availability_matrix = xr.DataArray(
        coords = {
            "bus": bus_list, 
            "y": cutout.grid.y.unique(),
            "x": cutout.grid.x.unique(),
        },
        dims = ["bus", "y","x"],
    )

    for bus in bus_list:
        cell_list = map_cell_to_region[map_cell_to_region["name"] == bus][["x","y"]]
        mask = availability_matrix.coords['x'].isin(cell_list['x']) & availability_matrix.coords['y'].isin(cell_list['y'])
        selected_data = building_area_in_cell.where(mask, drop=True)
        availability_matrix.loc[dict(bus=bus, x=selected_data.coords['x'], y=selected_data.coords['y'])] = (selected_data/selected_data.sum()).values

    availability_matrix.to_netcdf(f"availability/rooftop_availability_matrix_{region_num}.nc")

1
10


/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


27
34


/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


159


/home/pklein/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
